In [ ]:
from PIL import Image
from ultralytics import YOLO
import numpy as np
import pandas as pd
import cv2
import sys
import os
import time

In [ ]:
model = YOLO("yolov8s.pt") # COCO128 classes https://github.com/ultralytics/yolov5/blob/master/data/coco128.yaml
# model = YOLO("yolov8n-seg.pt")

In [ ]:
def detection(frames: np.array, should_save: bool = False) -> dict:
    results = model(frames, save=should_save)
    detected = {}

    for frame in range(len(results)):
        # print(frame, results[frame].boxes.numpy(), results[frame].boxes.cls)
        boxes = results[frame].boxes
        detected[frame] = []
        for box in boxes:
            box = box.numpy()
            # print(box, box.cls, box.conf)
            if int(box.cls[0]) == 0 and box.conf[0]> 0.7:
                detected[frame].append({
                    "xyxy": box.xyxy,
                    "xywh": box.xywh,
                    "conf": box.conf[0],
                    "center": box.xywh[0][0:2]
                })
        # print('\n')
    return detected

In [ ]:
def detection2(frames: np.array, should_save: bool = False) -> pd.DataFrame:
    results = model(frames, save=should_save)

    detected = [[x.boxes.numpy() for x in result if (x.boxes.cls == 0 and x.boxes.conf > 0.7)] for result in results]
    # print(detected)

    d = pd.Series([[i, det.xyxy[0], det.xywh[0], det.conf[0], det.xywh[0][0:2]] for i in range(len(detected)) for det in detected[i]])
    detected_bboxes = pd.DataFrame(d.values.tolist(), columns=["frame_nr", "xyxy", "xywh", "conf", "center"])

    return detected_bboxes

# detection2(frames)

In [ ]:
def distance(point1: np.array, point2: np.array) -> float:
    return np.linalg.norm(point1 - point2)

def is_closer_enough(dist: float, epsilon: float = 30) -> bool:
    return dist <= epsilon

def is_closer_enough2(dist: float, epsilon: float = 30) -> bool:
    return dist if dist <= epsilon else np.nan

In [ ]:
def mean_bbox(frames: dict) -> dict:
    same_bb = []
    for bbox0 in frames[0]: # bbox nel frame 0
        same_bb.append([])
        same_bb[-1].append(bbox0)
        # print(same_bbox)
        # print("0: ", bbox0["center"])

        for bbox1 in frames[1]: # bbox nel frame 1
            # print("\t1: ", bbox1["center"], "DIST: ", distance(bbox0["center"], bbox1["center"]))
            if is_closer_enough(distance(bbox0["center"], bbox1["center"])): # bbox0 <-> bbox1
                same_bb[-1].append(bbox1)
                # print("FOUND IT \t1: ", bbox1["center"])

        for bbox2 in frames[2]: # bbox nel frame 2
            # print("\t2: ", bbox2["center"], "DIST: ", distance(same_bbox[-1][-1]["center"], bbox2["center"]))
            if is_closer_enough(distance(same_bb[-1][-1]["center"], bbox2["center"])): # (bbox1 or bbox0) <-> bbox2
                same_bb[-1].append(bbox2)
                # print("FOUND IT \t2: ", bbox2["center"])

    # return same_bb
    # print(same_bb)
    mean_bb = []
    for bboxes in same_bb:
        bbs = {
            "xyxy": np.mean(np.array([frame["xyxy"] for frame in bboxes]), axis=0),
            "xywh": np.mean(np.array([frame["xywh"] for frame in bboxes]), axis=0),
            "conf": np.mean(np.array([frame["conf"] for frame in bboxes])),
            "center": np.mean(np.array([frame["center"] for frame in bboxes]), axis=0),
        }
        # print([frame["xyxy"] for frame in bboxes])
        # mean_bb.append(np.mean(np.array([frame["xyxy"] for frame in bboxes]), axis=0))
        mean_bb.append(bbs)

    return mean_bb

    # for frame_nr, frame in frames.items():
    #     print(frame_nr)
    #     for bbox in frame:
    #         print(bbox)

In [ ]:
def mean_bbox2(df: pd.DataFrame) -> pd.DataFrame:
    mean_bb = pd.DataFrame(columns=["xyxy", "xywh", "conf", "center"])

    # divide dataframe by frame nr
    frames = [df.loc[df["frame_nr"] == x].rename_axis("index").reset_index() for x in df["frame_nr"].unique()]
    # print(frames[0])

    f0_f1 = pd.merge(frames[0], frames[1], how="cross")
    f0_f1["distance"] = f0_f1.apply(lambda x: distance(x["center_x"], x["center_y"]), axis=1)
    f1_f2 = pd.merge(frames[1], frames[2], how="cross")
    f1_f2["distance"] = f1_f2.apply(lambda x: distance(x["center_x"], x["center_y"]), axis=1)
    f0_f2 = pd.merge(frames[0], frames[2], how="cross")
    f0_f2["distance"] = f0_f2.apply(lambda x: distance(x["center_x"], x["center_y"]), axis=1)

    same_bb_f0_f1 = f0_f1.loc[f0_f1["distance"].isin(f0_f1.groupby("index_x")["distance"].apply(lambda x: is_closer_enough2(x.min())))]
    same_bb_f1_f2 = f1_f2.loc[f1_f2["distance"].isin(f1_f2.groupby("index_x")["distance"].apply(lambda x: is_closer_enough2(x.min())))]
    same_bb_f0_f2 = f0_f2.loc[f0_f2["distance"].isin(f0_f2.groupby("index_x")["distance"].apply(lambda x: is_closer_enough2(x.min())))]

    # print(same_bb_f0_f1, "\n", same_bb_f1_f2, "\n", same_bb_f0_f2)

    concat = pd.concat([same_bb_f0_f1, same_bb_f1_f2, same_bb_f0_f2], ignore_index=True)
    # print(pd.merge(concat["index_x"], concat["index_y"], how="inner", left_on="index_x", right_on="index_y"))
    common_indexes = set(concat["index_x"]) & set(concat["index_y"]) # sono sia in index_x che in index_y

    bb_f0_f1 = concat.loc[concat["index_y"].isin(common_indexes)].drop("distance", axis=1)
    bb_f1_f2 = concat.loc[concat["index_x"].isin(common_indexes)].drop("distance", axis=1)
    bb_f1_f2.drop([col for col in bb_f1_f2.columns.values.tolist() if col.split('_')[-1] == 'x'], axis=1, inplace=True)
    bb_f1_f2 = bb_f1_f2.add_suffix("_z")
    bb_f0_f1_f2 = pd.concat([bb_f0_f1.reset_index(), bb_f1_f2.reset_index()], axis=1)
    bb_f0_f1_f2.drop("index", axis=1, inplace=True)

    bb_f0_f1_f2["xyxy"] = bb_f0_f1_f2.apply(lambda x: np.mean(np.array([x["xyxy_x"], x["xyxy_y"], x["xyxy_y_z"]]), axis=0), axis=1)
    bb_f0_f1_f2["xywh"] = bb_f0_f1_f2.apply(lambda x: np.mean(np.array([x["xywh_x"], x["xywh_y"], x["xywh_y_z"]]), axis=0), axis=1)
    bb_f0_f1_f2["conf"] = bb_f0_f1_f2.apply(lambda x: np.mean(np.array([x["conf_x"], x["conf_y"], x["conf_y_z"]])), axis=1)
    bb_f0_f1_f2["center"] = bb_f0_f1_f2.apply(lambda x: np.mean(np.array([x["center_x"], x["center_y"], x["center_y_z"]]), axis=0), axis=1)

    not_common_indexes = set(set(concat["index_x"]) - set(bb_f0_f1["index_x"]) - set(common_indexes))
    # print(not_common_indexes)

    bb_f0_f2 = concat.loc[concat["index_x"].isin(not_common_indexes)].drop("distance", axis=1)
    bb_f0_f2["xyxy"] = bb_f0_f2.apply(lambda x: np.mean(np.array([x["xyxy_x"], x["xyxy_y"]]), axis=0), axis=1)
    bb_f0_f2["xywh"] = bb_f0_f2.apply(lambda x: np.mean(np.array([x["xywh_x"], x["xywh_y"]]), axis=0), axis=1)
    bb_f0_f2["conf"] = bb_f0_f2.apply(lambda x: np.mean(np.array([x["conf_x"], x["conf_y"]])), axis=1)
    bb_f0_f2["center"] = bb_f0_f2.apply(lambda x: np.mean(np.array([x["center_x"], x["center_y"]]), axis=0), axis=1)

    # mean_bb["index_0"] = bb_f0_f1_f2["index_x"]
    # mean_bb["index_1"] = bb_f0_f1_f2["index_y"]
    # mean_bb["index_2"] = bb_f0_f1_f2["index_y_z"]
    # mean_bb["frame_nr_0"] = bb_f0_f1_f2["frame_nr_x"]
    # mean_bb["frame_nr_1"] = bb_f0_f1_f2["frame_nr_y"]
    # mean_bb["frame_nr_2"] = bb_f0_f1_f2["frame_nr_y_z"]
    mean_bb["xyxy"] = pd.concat([bb_f0_f1_f2["xyxy"], bb_f0_f2["xyxy"]], axis=0, ignore_index=True)
    mean_bb["xywh"] = pd.concat([bb_f0_f1_f2["xywh"], bb_f0_f2["xywh"]], axis=0, ignore_index=True)
    mean_bb["conf"] = pd.concat([bb_f0_f1_f2["conf"], bb_f0_f2["conf"]], axis=0, ignore_index=True)
    mean_bb["center"] = pd.concat([bb_f0_f1_f2["center"], bb_f0_f2["center"]], axis=0, ignore_index=True)

    return mean_bb

In [ ]:
path = "./runs2/detect/predict1/og/image%d.jpg"
frames = [Image.open(path % 0), Image.open(path % 1), Image.open(path % 2)]

start = time.time()

det = detection2(frames, False)

# det
d1 = det["center"].tolist()
d2 = det["center"].tolist()[:-1]

# d2



out = []



out = np.concatenate((a, np.array(d1)), axis=0)
print(a)
np.concatenate((a, np.array(d2)), axis=0)

# np.append(d1)
#
# [[[]], [[]]]
# [[], []]
# np.concatenate((np.array(d1), np.array(d2)), axis=0)


In [ ]:

mean_bboxes = mean_bbox2(det)
# det

# for xyxy in mean_bboxes["xyxy"]:
#     print(xyxy)
#     cropped = frames[0].crop(xyxy)
#     resized = cropped.resize((250, 250))
#     display(resized)

print(time.time() - start)

start2 = time.time()

det2 = detection(frames, False)
mean_bboxes2 = mean_bbox(det2)
# det

# for xyxy in mean_bboxes["xyxy"]:
#     print(xyxy)
#     cropped = frames[0].crop(xyxy)
#     resized = cropped.resize((250, 250))
#     display(resized)

print(time.time() - start2)

In [ ]:
file_path = 'test.mp4'
delay = 1
window_name = 'frame'

cap = cv2.VideoCapture(file_path)
# cap = cv2.VideoCapture(0)

if not cap.isOpened():
    sys.exit()

count = 3
# frames = []
round = 0
while round < 5:
# while True:

    reads = [cap.read() for i in range(count)] # [(True, [...]] if frame read else [(False, None)]
    frames = [read[1] for read in reads if read[0]]
    rets = [read[0] for read in reads]

    # os.makedirs("./runs/detect/predict/og/")
    frames_to_detect = list(frames)
    det = detection2(frames_to_detect, should_save=True)
    mean_bboxes = mean_bbox2(det)

    for i in range(len(frames)):
        if rets[i]:
            os.makedirs("./runs/detect/predict/og_f%d/" % i)
            cv2.imwrite("./runs/detect/predict/og_f%d/image.jpg" %i, frames[i])
            for index, row in mean_bboxes.iterrows():
                cropped = Image.fromarray(cv2.cvtColor(frames[i], cv2.COLOR_BGR2RGB)).crop(row["xyxy"])
                # resized = cropped.resize((250, 250))
                cropped.save(f"./runs/detect/predict/og_f{i}/cropped{index}.jpg")
                # cv2.imwrite(f"./runs/detect/predict/og_f{i}/cropped{index}.jpg", resized)

    os.rename("./runs/detect/predict", "./runs/detect/predict%d" % round)

    # ret, frame = cap.read()
    # frames.append(frame)
    # count -= 1
    # print(count, frames)
    #
    # if count == 0:
    #     # print(frames)
    #     os.makedirs("./runs/detect/predict/og/")
    #     for index in range(len(frames)):
    #         cv2.imwrite("./runs/detect/predict/og/image%d.jpg" % index, frames[index])
    #
    #     bbs = detection(frames)
    #     os.rename("./runs/detect/predict", "./runs/detect/predict%d" % round)
    #
    #     count = 3
    #     frames = []

    round += 1

    if rets[2]:
        # cv2.imshow(window_name, frame)
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
    else:
        # os.makedirs("./runs/detect/predict/og/")
        # for index in range(len(frames)):
        #     cv2.imwrite("./runs/detect/predict/og/image%d.jpg" % index, frames[index])
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        break

cap.release()
cv2.destroyWindow(window_name)

In [ ]:
path = "./runs2/detect/predict4/og/image%d.jpg"
frames = [Image.open(path % 0), Image.open(path % 1), Image.open(path % 2)]

start = time.time()
detected = detection(frames)
# detected
mean_bboxes = mean_bbox(detected)
# mean_bboxes

for meanbb in mean_bboxes:
    bbox = meanbb["xyxy"][0].astype(int).tolist()
    # bboxwh = meanbb["xywh"][0].astype(int).tolist()
    cropped = frames[0].crop(bbox)
    resized = cropped.resize((250, 250))
    display(resized)

print(time.time() - start)

In [ ]:
bbs = detection([image, image, image], True)
#
# print(bbs)

result = Image.open('./runs/detect/predict/image0.jpg')
display(result)

for bb in bbs[0]:
    bbox = bb["xyxy"][0].astype(int).tolist()
    bboxwh = bb["xywh"][0].astype(int).tolist()
    cropped = image.crop(bbox)
    resized = cropped.resize((250, 250))
    display(resized)

bbs

# for _, bb in bbs[0].items():
#     for bbox in bb:
#         bbox = bbox.xyxy[0].astype(int).tolist()
#
#         cropped = image.crop(bbox)
#         resized = cropped.resize((250, 250))
#         display(resized)

In [ ]:
file_path = 'test.mp4'
delay = 1
window_name = 'frame'

cap = cv2.VideoCapture(file_path)
# cap = cv2.VideoCapture(0)

if not cap.isOpened():
    sys.exit()

count = 3
frames = []
round = 0
# while round < 20:
while True:
    ret, frame = cap.read()
    frames.append(frame)
    count -= 1
    # print(count, frames)

    if count == 0:
        # print(frames)
        os.makedirs("./runs/detect/predict/og/")
        for index in range(len(frames)):
            cv2.imwrite("./runs/detect/predict/og/image%d.jpg" % index, frames[index])

        bbs = detection(frames)
        os.rename("./runs/detect/predict", "./runs/detect/predict%d" % round)

        count = 3
        frames = []
        round += 1

    if ret:
        # cv2.imshow(window_name, frame)
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
    else:
        os.makedirs("./runs/detect/predict/og/")
        for index in range(len(frames)):
            cv2.imwrite("./runs/detect/predict/og/image%d.jpg" % index, frames[index])
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        break

cap.release()
cv2.destroyWindow(window_name)

In [ ]:
results = model([image], save=True)
# results = results
print(results)

boxes, masks, probs = ([], [], [])
for result in results:
    # print(result)
    boxes = result.boxes.numpy()
    masks = result.masks
    probs = result.probs

# print("Boxes")
print(boxes)
# print("Masks")
# print(masks)
# print("Probs")
# print(probs)

In [ ]:
for bbox in boxes:
    bbox = bbox.xyxy[0].astype(int).tolist()

    cropped = image.crop(bbox)
    resized = cropped.resize((250, 250))
    # display(resized)

In [ ]:
# Image.open('runs/detect/predict/image0.jpg')
# Image.open('runs/segment/predict/image0.jpg')